In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
import joblib
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [8]:
with tf.device('/GPU:0'):
  file = "/content/drive/My Drive/dataset_6k.csv"
  dataset = pd.read_csv(file)

  X = dataset.drop('label', axis=1)
  y = dataset['label']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

  X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

  X_train_np = X_train.values
  X_test_np = X_test.values
  X_val_np = X_val.values

  X_train = X_train_np.reshape(X_train.shape[0], X_train.shape[1], 1)
  X_test = X_test_np.reshape(X_test.shape[0], X_test.shape[1], 1)
  X_val = X_val_np.reshape(X_val.shape[0], X_test.shape[1], 1)

  classes = np.array([0,1])

  class_weights = compute_class_weight('balanced', classes=classes, y=y_train)

In [12]:
# Define the CNN architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=[X_train.shape[1], 1]),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    # tf.keras.layers.Conv1D(64, 3, activation='relu'),
    # tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with class weights
model.fit(X_train, y_train, epochs=25, batch_size=1000, validation_data=(X_test, y_test))

saved_model = joblib.dump(model, "/content/drive/My Drive/CNN_model.joblib")

loss, accuracy = model.evaluate(X_test, y_test)

# prediction = model.predict(X_val)
# result = accuracy_score(y_val, prediction)

print("Loss: {}, Accuracy: {}".format(loss, accuracy))
# print("Prediction on validation set: {}".format(result))

Epoch 1/25
4/4 [==============================] - 3s 582ms/step - loss: 88575.9219 - accuracy: 0.4936 - val_loss: 8963.6230 - val_accuracy: 0.4921
Epoch 2/25
4/4 [==============================] - 1s 360ms/step - loss: 2451.6113 - accuracy: 0.7550 - val_loss: 0.6862 - val_accuracy: 0.5179
Epoch 3/25
4/4 [==============================] - 1s 388ms/step - loss: 0.6926 - accuracy: 0.4917 - val_loss: 0.6931 - val_accuracy: 0.5079
Epoch 4/25
4/4 [==============================] - 1s 358ms/step - loss: 0.6932 - accuracy: 0.4908 - val_loss: 0.6931 - val_accuracy: 0.5079
Epoch 5/25
4/4 [==============================] - 1s 366ms/step - loss: 0.6933 - accuracy: 0.4908 - val_loss: 0.6931 - val_accuracy: 0.5079
Epoch 6/25
4/4 [==============================] - 2s 501ms/step - loss: 0.6933 - accuracy: 0.4908 - val_loss: 0.6931 - val_accuracy: 0.5079
Epoch 7/25
4/4 [==============================] - 2s 527ms/step - loss: 0.6933 - accuracy: 0.4908 - val_loss: 0.6931 - val_accuracy: 0.5079
Epoch 8/25